## Importações

In [10]:
import pandas as pd
from math import sqrt

ROOT = "dataset processado"
RATINGS_ONLY_PATH = ROOT + '/ratings-only.csv'
MOVIES_ONLY_PATH = ROOT + '/movies-only.csv'

## ratings-only.csv para dicionário users

In [11]:
subset = pd.read_csv(RATINGS_ONLY_PATH)
subset.head()

,userId,movieId,movieTitle,rating
0,15,1,Toy Story,2.0
1,15,2,Jumanji,2.0
2,15,5,Grumpier Old Men,4.5
3,15,6,Waiting to Exhale,4.0
4,15,10,Father of the Bride Part II,3.0


In [12]:
users = dict()
for i in range(len(subset)):
  id = str(subset.iloc[i]['userId'])
  if id not in users.keys():
    users[id] = dict()
  users[id][f'''{subset.iloc[i]['movieTitle']}'''] = subset.iloc[i]['rating']
users

{'15': {'Toy Story': 2.0,
  'Jumanji': 2.0,
  'Grumpier Old Men': 4.5,
  'Waiting to Exhale': 4.0,
  'Father of the Bride Part II': 3.0,
  'Tom and Huck': 2.5,
  'Sudden Death': 2.5,
  'GoldenEye': 3.5,
  'The American President': 3.0,
  'Dracula: Dead and Loving It': 1.0,
  'Balto': 4.5,
  'Nixon': 2.5,
  'Cutthroat Island': 3.0,
  'Casino': 4.0,
  'Four Rooms': 3.0,
  'Ace Ventura: When Nature Calls': 1.0,
  'Money Train': 2.5,
  'Get Shorty': 3.0,
  'Copycat': 5.0,
  'Assassins': 5.0,
  'Powder': 2.5,
  'Leaving Las Vegas': 2.0,
  'Now and Then': 0.5,
  'The City of Lost Children': 5.0,
  'Shanghai Triad': 3.0,
  'Dangerous Minds': 1.5,
  'Twelve Monkeys': 4.0,
  'Wings of Courage': 1.0,
  'Babe': 2.0,
  'Carrington': 3.0,
  'Dead Man Walking': 5.0,
  'Across the Sea of Time': 2.5,
  'Clueless': 4.0,
  'Dead Presidents': 3.5,
  'Mortal Kombat': 3.5,
  'To Die For': 5.0,
  'How To Make An American Quilt': 3.0,
  'Se7en': 1.0,
  'Pocahontas': 2.0,
  'When Night Is Falling': 0.5,
  'Th

## Funções básicas de filtragem



In [13]:
''' Computa a similaridade do cosseno.    
Rating1 e rating2 são dicionários na forma
{'The Strokes': 3.0, 'Slightly Stoopid': 2.55} '''

def cosseno(rating1, rating2):
    xlen = 0
    ylen = 0
    numerator = 0

    for key in rating1:
        if key in rating2:
          potency1 = pow(rating1[key], 2)
          xlen += potency1
          '''
          print(f'Item {key}: ao quadrado para x = {potency1}')
          '''

          potency2 = pow(rating2[key], 2)
          ylen += potency2
          '''
          print(f'Item {key}: ao quadrado para y = {potency2}')
          '''

          parcel = rating1[key]*rating2[key]
          numerator += parcel
          '''
          print(f'Item {key}: parcela x * y = {parcel}\n')
          '''
    '''      
    print(f'NUMERADOR = {numerator}\n')
    '''

    denominator = sqrt(xlen) * sqrt(ylen)
    '''
    print(f'DENOMINADOR:')
    print(f'Comprimento do vetor x = {xlen}')
    print(f'Quadrado do comprimento do vetor x = {sqrt(xlen)}')
    print(f'Comprimento do vetor y = {ylen}')
    print(f'Quadrado do comprimento do vetor y = {sqrt(ylen)}')
    print(f'Produto dos quadrados = {denominator}\n')
    '''
    
    if denominator == 0:
        return 0
    else:
        return round(numerator / denominator, 5)

print(cosseno(users['15'], users['232']))

0.91724


In [14]:
'Cria uma lista de usuários ordenada pela distancia deles do usuário alvo'

def computeNearestNeighbors(target, users):    
    distances = []
    for user in users:
        if user != target:
            distance = cosseno(users[user], users[target])
            # print(distance)
            distances.append((distance, user))
    distances.sort() # ordena pela menor distancia
    return distances

In [15]:
'Retorna uma lista de recomendações'

def recommend(target, users):
    # encontra o vizinho mais próximo do usuário alvo
    nearest = computeNearestNeighbors(target, users)[0][1]

    recommendations = []
    neighborRatings = users[nearest] # avaliações do vizinho
    targetRatings = users[target] # avaliações do alvo

    # encontrando itens que o vizinho avaliou, mas o alvo não
    for key in neighborRatings:
        if not key in targetRatings:
            item = {
                'movieTitle': key,
                'rating': neighborRatings[key],
            }
            recommendations.append(item)
    recommendations = pd.DataFrame(columns=["movieTitle", "rating"], data=recommendations)

    # usando o fn classificado para variedade - a classificação é mais eficiente
    return recommendations.sort_values(by=['movieTitle'], ascending=True)

recommendations = recommend('232', users)
recommendations.head(15)

,movieTitle,rating
185,1-900,4.0
332,101 Dalmatians,1.5
741,28 Days,2.0
51,3 Ninjas Kick Back,2.5
551,8 Women,2.5
387,976-EVIL,0.5
425,A Brother's Kiss,4.0
426,A Chef in Love,2.5
203,A Couch in New York,3.0
246,A Cry in the Dark,2.5


In [16]:
df_movies = pd.read_csv(MOVIES_ONLY_PATH)
df_movies.head()

,currentId,title,overview,genres,runtime,releaseDate,posterPath
0,1,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Animation, Comedy, Family",81.0,1995-10-30,https://flixpatrol.com/runtime/cache/files/pos...
1,2,Jumanji,When siblings Judy and Peter discover an encha...,"Adventure, Fantasy, Family",104.0,1995-12-15,https://flixpatrol.com/runtime/cache/files/pos...
2,5,Grumpier Old Men,A family wedding reignites the ancient feud be...,"Romance, Comedy",101.0,1995-12-22,https://flixpatrol.com/runtime/cache/files/pos...
3,6,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","Comedy, Drama, Romance",127.0,1995-12-22,https://flixpatrol.com/runtime/cache/files/pos...
4,10,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,106.0,1995-02-10,https://flixpatrol.com/runtime/cache/files/pos...


In [17]:
def get_formatted_recommendations(recommendations):
  df_movies = pd.read_csv(MOVIES_ONLY_PATH)
  df_movie_ratings = df_movies[df_movies['title'].isin(recommendations['movieTitle'])]
  df_movie_ratings = df_movie_ratings.sort_values(by=['title'], ascending=True).set_index(recommendations.index)
  df_movie_ratings.insert(2, 'rating', recommendations['rating'])
  df_movie_ratings = df_movie_ratings.sort_values(by=['rating'], ascending=False)
  return df_movie_ratings

In [18]:
df_formatted_recommendations = get_formatted_recommendations(recommendations)
df_formatted_recommendations.head(15)

,currentId,title,rating,overview,genres,runtime,releaseDate,posterPath
594,72998,Mortal Kombat: Annihilation,5.0,A group of heroic warriors has only six days t...,"Action, Fantasy",95.0,1997-11-21,https://flixpatrol.com/runtime/cache/files/pos...
199,4016,The Wild McCullochs,5.0,"A story about the rich McCulloch Family, their...","Romance, Drama",93.0,1975-05-21,https://flixpatrol.com/runtime/cache/files/pos...
421,44199,Keys to Tulsa,5.0,Richter Boudreau is on a bad streak: Languishi...,"Crime, Drama, Thriller",113.0,1997-04-11,https://flixpatrol.com/runtime/cache/files/pos...
350,8644,Bastard Out of Carolina,5.0,"Difficult tale of poor, struggling South Carol...",Drama,97.0,1996-12-15,https://flixpatrol.com/runtime/cache/files/pos...
562,68358,Gattaca,5.0,Science fiction drama about a future society i...,"Thriller, Mystery, Romance",106.0,1997-09-07,https://flixpatrol.com/runtime/cache/files/pos...
762,112852,Friday the 13th Part VI: Jason Lives,5.0,"As a child, Tommy killed mass-murderer Jason. ...","Horror, Mystery, Thriller",86.0,1986-08-01,https://flixpatrol.com/runtime/cache/files/pos...
627,81229,The Education of Little Tree,5.0,"Little Tree is an 8-year-old Cherokee boy, who...",Drama,115.0,1997-12-25,https://flixpatrol.com/runtime/cache/files/pos...
671,89745,The Newton Boys,5.0,Four Newton brothers are a poor farmer family ...,"Crime, Action, Drama",113.0,1998-03-26,https://flixpatrol.com/runtime/cache/files/pos...
305,6754,A Room with a View,5.0,When Lucy Honeychurch and chaperon Charlotte B...,"Drama, Romance",117.0,1985-12-13,https://flixpatrol.com/runtime/cache/files/pos...
213,4343,Dumbo,5.0,Dumbo is a baby elephant born with oversized e...,"Animation, Family",64.0,1941-10-22,https://flixpatrol.com/runtime/cache/files/pos...
